# AI for Ethics
Final Project Summer 2020

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import sys
!{sys.executable} -m pip install aif360
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install cvxpy

     |████████████████████████████████| 818 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 15.5 MB/s eta 0:00:01


     |████████████████████████████████| 3.5 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 21.3 MB/s eta 0:00:01
  Created wheel for ecos: filename=ecos-2.0.7.post1-cp37-cp37m-macosx_10_9_x86_64.whl size=84254 sha256=ad8e9d4fd886b93e1a14d8c40e06ef1b0a33fa515b8b45287d3acbe1092a5945
  Stored in directory: /Users/jagrusy/Library/Caches/pip/wheels/70/e7/fd/4a4f957fc18a0f0407ca4b331c51e6a469c92efec58f732edf
  Created wheel for scs: filename=scs-2.1.2-cp37-cp37m-macosx_10_15_x86_64.whl size=119364 sha256=acc0c54d1ec5187c365f14aac18d2cc3f2de31efaf0e8d1599f32f2c5113d32d
  Stored in directory: /Users/jagrusy/Library/Caches/pip/wheels/bc/e9/2d/41c52bfaefbbbee5b3c6c189e168039f6ac9b40fa2e14fe96d
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=8d738e696e83eb0b8fd41a6a6f796fa5f09dd5a1e1c1619b2cbac1e7e281b88e
  Stored in directory: /Users/jagrusy/Library/Caches/pip/whee

In [5]:
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import CompasDataset

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing import Reweighing
# from aif360.algorithms.preprocessing import DisparateImpactRemover
# import BlackBoxAuditing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

from IPython.display import Markdown, display


In [4]:
dataset = CompasDataset()
dataset

               instance weights            features                            \
                                protected attribute       protected attribute   
                                                sex   age                race   
instance names                                                                  
1                           1.0                 0.0  69.0                 0.0   
3                           1.0                 0.0  34.0                 0.0   
4                           1.0                 0.0  24.0                 0.0   
7                           1.0                 0.0  44.0                 0.0   
8                           1.0                 0.0  41.0                 1.0   
...                         ...                 ...   ...                 ...   
10996                       1.0                 0.0  23.0                 0.0   
10997                       1.0                 0.0  23.0                 0.0   
10999                       

In [6]:
display(Markdown("### Measuring Fairness and Mitigating Bias for Race"))

privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]
dataset_orig = load_preproc_data_compas(['race'])

dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

### Measuring Fairness and Mitigating Bias for Race

In [7]:
optim_options = {
    "distortion_fun": get_distortion_compas,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

In [7]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.130476


In [9]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train_rw = RW.fit_transform(dataset_orig_train)

OP = OptimPreproc(OptTools, optim_options,
                  unprivileged_groups = unprivileged_groups,
                  privileged_groups = privileged_groups)

OP = OP.fit(dataset_orig_train)

# Transform training data and align features
dataset_transf_train_op = OP.transform(dataset_orig_train, transform_Y=True)
dataset_transf_train_op = dataset_orig_train.align_datasets(dataset_transf_train_op)

/Users/jagrusy/opt/miniconda3/lib/python3.7/site-packages/aif360/algorithms/preprocessing/optim_preproc.py:68: UserWarning: Privileged and unprivileged groups specified will not be used. The protected attributes are directly specified in the data preprocessing function. The current implementation automatically adjusts for discrimination across all groups. This can be changed by changing the optimization code.
  warn("Privileged and unprivileged groups specified will not be "
/Users/jagrusy/opt/miniconda3/lib/python3.7/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/Users/jagrusy/opt/miniconda3/lib/pytho

Optimized Preprocessing: Objective converged to 0.000009


In [10]:
metric_transf_train_rw = BinaryLabelDatasetMetric(dataset_transf_train_rw, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset using Reweighing"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train_rw.mean_difference())

metric_transf_train_op = BinaryLabelDatasetMetric(dataset_transf_train_op, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset using Optimized Preprocessing"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train_op.mean_difference())

#### Transformed training dataset using Reweighing

Difference in mean outcomes between unprivileged and privileged groups = -0.000000


#### Transformed training dataset using Optimized Preprocessing

Difference in mean outcomes between unprivileged and privileged groups = -0.052732


In [11]:
display(Markdown("### Measuring Fairness and Mitigating Bias for Sex"))

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_compas(['sex'])

### Measuring Fairness and Mitigating Bias for Sex

In [12]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train_rw = RW.fit_transform(dataset_orig_train)

OP = OptimPreproc(OptTools, optim_options,
                  unprivileged_groups = unprivileged_groups,
                  privileged_groups = privileged_groups)

OP = OP.fit(dataset_orig_train)

# Transform training data and align features
dataset_transf_train_op = OP.transform(dataset_orig_train, transform_Y=True)
dataset_transf_train_op = dataset_orig_train.align_datasets(dataset_transf_train_op)

ValueError: 'sex' is not in list

In [10]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(3694, 10)


#### Favorable and unfavorable labels

0.0 1.0


#### Protected attribute names

['race']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


In [11]:
df = pd.read_csv(filepath_or_buffer='./compas-scores-two-years.csv')

In [12]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [13]:
df.shape

(7214, 53)

In [14]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')